## Attention 신경망 구현 및 학습

In [43]:
import random
import tensorflow as tf
from konlpy.tag import Okt

In [44]:
EPOCHS = 200
NUM_WORDS = 2000

## Encoder

In [45]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)  # Embedding : one-hot 인코딩을 실수 값으로 바꿈.
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)  # return_state를 True로 해야 lstm을 했을때 h와 c의 출력이 나온다.
        
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)  # H : 모든 hidden state (Key, Value)
        return H, h, c   # 입력이 들어오면 lstm을 통해 h, c를 context로 출력.

## Decoder

In [46]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)   # 학습을 하는 과정에서는 shifted output
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)  # return_sequences=True : True로 하지 않으면 마지막 x 하나만 return하게 된다.
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')   # fully connected layer

    def call(self, inputs, training=False, mask=None):
        x, s0, c0, H = inputs  
        # x : shifted output  # s0 : first decoder hidden state, c0 : first cell state, H : all encoder hidden state
        x = self.emb(x)
        S, h, c = self.lstm(x, initial_state=[s0, c0])   # S : 모든 hidden state => 얘를 Query로 사용, H : Key와 Value로 사용

        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)  # S_는 하나 앞 선 tiem_step
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)

        return self.dense(y), h, c

## Seq2seq

In [47]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs # 학습을 하기 위해선 x 뿐만 아니라 y(출력)도 알아야된다. ==> decoder에서 shifted output을 넣어주기 위해서.
            H, h, c = self.enc(x)  # input을 넣어서 hidden state, cell state 출력.
            y, _, _ = self.dec((y, h, c, H))  # hidden state와 cell state를 초기값으로 입력받고 shifted ouyput y를 입력으로 받는 decoder
            return y  # y : decoder 결과 => 전체 문장
        else:   # test
            x = inputs
            H, h, c = self.enc(x)

            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):  # tf.keras.Model에 의해서 call 함수가 autograph로 변환. => tf for문 => tf를 써줘야한다.
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)  # y : softmax를 통과한 것이 마지막 출력으로 나오기 때문에 one-hot 벡터로 바꿔주고 sparse표현으로 바꾸기 위해서 argmax
                y = tf.reshape(y, (1, 1))  # rashape를 하는 이유 : dim이 하나밖에 없기 때문. Batch를 고려해줘야 하기 때문에 batch가 하나라는 걸 표현.
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))   # stack : TensorArray로 받은 것들을 쌓아 올린다. (1 : batch size, 64 : 문장의 최대 길이)

## 학습, 테스트 루프 정의

In [48]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [49]:
dataset_file = 'chatbot_data.csv'  # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [50]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [51]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    if epoch % 20 == 0:
      print(template.format(epoch + 1,
                            train_loss.result(),
                            train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 2.997462749481201, Accuracy: 83.08662414550781
Epoch 21, Loss: 0.4149985611438751, Accuracy: 92.55168914794922
Epoch 41, Loss: 0.33815595507621765, Accuracy: 93.3192367553711
Epoch 61, Loss: 0.23589947819709778, Accuracy: 94.60369873046875
Epoch 81, Loss: 0.10030584037303925, Accuracy: 97.81484985351562
Epoch 101, Loss: 0.03481760993599892, Accuracy: 99.5614013671875
Epoch 121, Loss: 0.016270585358142853, Accuracy: 99.83943939208984
Epoch 141, Loss: 0.007010623812675476, Accuracy: 99.91776275634766
Epoch 161, Loss: 0.004633449949324131, Accuracy: 99.92559814453125
Epoch 181, Loss: 0.003663546172901988, Accuracy: 99.91776275634766


## 테스트 루프

In [52]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['영수증 이 입니다 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['네 진동 벨 울리면 픽업 테이블 로 와주세요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['총 8800원 입니다 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 총 주문 건 없으세요 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 가능합니다 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 포인트 사용 카드 도 와 드리겠습니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 그럼 청소 건 아닌데요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['4000원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['10분 정도 걸려요 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['루이보스 아쌈 이렇게 두 개 이 다 더 있습니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['들어가있습니다 \n']
_
q:  ['오늘 의 커피 는 커피 로 하나요 맛 이 \n'